<a href="https://colab.research.google.com/github/ssanchezgoe/curso_deep_learning_economia/blob/main/NBs_Google_Colab/DL_S12_Regularizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="140px" src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" align="left" hspace="10px" vspace="0px"></p>

<h1> Curso Deep Learning: Economía</h1>

## S12: Regularización

In [ ]:
# Librerías
from tensorflow import keras
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np

In [ ]:
#@title Funciones a usar
#@markdown plot_decision_boundary()
def plot_decision_boundary(model, X, y):
    amin, bmin = X.min(axis=0) - 0.1
    amax, bmax = X.max(axis=0) + 0.1
    hticks = np.linspace(amin, amax, 101)
    vticks = np.linspace(bmin, bmax, 101)
    
    aa, bb = np.meshgrid(hticks, vticks)
    ab = np.c_[aa.ravel(), bb.ravel()]
    
    c = model.predict(ab)
    cc = c.reshape(aa.shape)

    plt.figure(figsize=(12, 8))
    plt.contourf(aa, bb, cc, cmap='bwr', alpha=0.2)
    plt.plot(X[y==0, 0], X[y==0, 1], 'ob', alpha=0.5)
    plt.plot(X[y==1, 0], X[y==1, 1], 'xr', alpha=0.5)
    plt.legend(['0', '1'])


# Regularización

## Introducción

Muchas veces, cuando entrenamos nuestro modelo y procedemos a validarlo, encontramos que éste no puede generalizar de una forma correcta, es decir puede presentar **sobreajuste**, **alta varianza** u  **overfitting**. Existen multiples razones por las cuales esto se dé, poca cantidad de datos, una mala elección del modelo y su complejidad, entre otros.

También se puede dar el caso contrario en que el modelo que se cree sea muy general y, aunque el desempeño en el conjunto de entrenamiento y evaluación es similar, el puntaje de clasificación es muy bajo. En este caso, decimos que el modelo sufre de **subajuste**, **alto sesgo** o **underfitting**.

La siguiente gráfica resume los dos problemas mencionados hasta ahora

<p><img alt="Colaboratory logo" height="200px" src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Screen-Shot-2018-04-03-at-7.52.01-PM-e1522832332857.png" align="center" hspace="10px" vspace="0px"></p>

Una forma de *evitar el overfitting es regularizando nuestro modelo*, es decir poniendo ciertas restricciónes a los pesos $W$, de forma tal que se fuercen a tomar valores pequeños, lo cual hará que su distribución de valores más regular. Como en el caso de las regresiones lineales, una de las formas de regularizar un modelo consiste en añadir un término a la función de costo, el cual, de acuerdo a su, define el tipo de regularización (**Ridge, lasso, Elasticnet**); o bien deteniendo el entrenamiento cuando el error de validación alcanzaba el mínimo, **Early stopping**. El *Early stopping* corresponde a una forma de hallar el valor optimo del número de épocas para produccir un modelo sin sobreajuste.

Veamos cómo se puede usar los diferentes tipos de regularización y cómo se puede realizar su implementación en redes neuronales profundas.

## **Regularización L1 (Lasso) y L2() Ridge**

Los métodos más simples de regularización son $l1$ y $l2$, los cuales consisten en añadir un término a nuestra función de costo, el cual dependerá de cual de las regularizaciones usemos. Así, se castigan los valores grandes de la matriz de pesos y se restringen a pequeños valores. De forma aproximada, la regularización tiene la siguiente forma:

\begin{equation}
J(W,b)=\frac{1}{m}\sum_{i}^{m}L(\hat{y}^{(i)},y^{(i)}) + \text{regularización}(W)
\end{equation}

Como vemos la regularización se realizará sobre los pesos de la red sin incluir el término de sesgo o inclinación $b$ (bias term), esto se debe a que la mayoría de valores se encuentran en $W$. Lo anterior no impide a que podamos hacer también una penalización sobre los términos de sesgo.

Para el caso de regularizasción **Rigida o $l2$** tenemos la siguiente forma para el costo

\begin{equation}
J(W,b)=\frac{1}{m}\sum_{i}^{m}L(\hat{y}^{(i)},y^{(i)}) + \frac{\lambda}{2m}\sum_{l=1}^{L}||W^{[l]}||^{2}_{F}
\end{equation}

Donde el término de regularización es una suma sobre $l2$ norma, que consisten en elevar al cuadrado las componentes de la matriz de pesos, a esta forma de regularización también se le conoce como **weight decay**.

De forma similar, para la regularización **lasso o $l1$** se tiene

\begin{equation}
J(W,b)=\frac{1}{m}\sum_{i}^{m}L(\hat{y}^{(i)},y^{(i)}) + \frac{\lambda}{2m}\sum_{l=1}^{L}||W^{[l]}||
\end{equation}

donde la suma se da sobre los valores absolutos de las componentes de la matriz de pesos. Como hemos mencionado la regularización busca disminuir y evitar el overfitting, el cual puede presentarse en parte debido a la complejidad de nuestro modelo (ver siguiente ilustración).

<p><img alt="Colaboratory logo" height="250px" src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Screen-Shot-2018-04-04-at-1.53.35-AM.png" align="center" hspace="10px" vspace="0px"></p>

Como vemos $l1$ y $l2$ consisten en añadir terminos proporcionales a $\lambda$ (un hiperparámetro), que de acuerdo a su valor va a restringir nuestros pesos a ciertos pequeños valores. Recordemos que 

\begin{equation}
Z=W^{T}X + b
\end{equation}

Si se tiene pesos pequeños, esto implicará un $Z$ pequeño, debido a su proporcionalidad, y por lo tanto el efecto de la función de activación estará restringido a valores pequeños, lo cual podemos traducir como una disminución en la complejidad del modelo,que es lo buscado.

<p><img alt="Colaboratory logo" height="250px" src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Screen-Shot-2018-04-04-at-1.53.40-AM.png" align="center" hspace="10px" vspace="0px"></p>

De nuevo, esta calibración será guiada por el hiperparámetro $\lambda$, pues de no tener un valor correcto se puede traducir en underfitting, cuando lo que se busca es el mejor modelo posible.

<p><img alt="Colaboratory logo" height="250px" src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Screen-Shot-2018-04-03-at-10.37.28-PM.png" align="center" hspace="200px" vspace="0px"></p>

`keras` proporciona 3 formas de realizar regularización:

1. A través del parámetro `l1`: suma absoluta de pesos.
2. A través del parámetro `l2`: suma cuadrática de pesos.
3. A través del parámetro `l1_12`: sima absoluta y cuadrática de pesos (*elastic-net*).


Cuando usamos keras, una forma de llamar el regularizador es a través del parámetro de la capa `kernel_regularizer`, de la siguiente forma:

In [ ]:
layerl1 = keras.layers.Dense(10, activation="relu",kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l1(0.01),bias_regularizer='l1')

layerl2 = keras.layers.Dense(10, activation="relu",kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l2(0.01),bias_regularizer='l2')

layerl1_l2 = keras.layers.Dense(10, activation="relu",kernel_initializer="he_normal",
                            kernel_regularizer=keras.regularizers.l1_l2(0.01),bias_regularizer='l1_l2')

El argumento de las funciones `keras.regularizers.l1()`, `keras.regularizers.l2()` y `keras.regularizers.l1_l2()` corresponde al valor $\lambda$ descrito en las ecuaciones de regularización.

## Ejemplo guiado

En este ejemplo se demostrará la regularización de pesos para reducir el overfitting en un problema de clasificación simple.

**Problema:** Usaremos una clasificación binaria en un conjunto de datos definido por dos semilunas (una semiluna por cada clase)

In [ ]:
X, y = make_moons(n_samples=100, noise=0.2, random_state=1)
df = DataFrame(dict(x1=X[:,0], x2=X[:,1], label=y))
df

Podemos usar el método `groupby` de pandas para agrupar los conjuntos por etiquetas (0 y 1) y graficar cada subgrupo con un color:

In [ ]:
for key, group in df.groupby('label'):
  print(key)
  print(group)

In [ ]:
colors = {0:'red', 1:'blue'}
fig, ax = plt.subplots()
df.groupby('label')
for key, group in df.groupby('label'):
    group.plot(ax=ax, kind='scatter', x='x1', y='x2', label=key, color=colors[key])
plt.show()

Este problema de pruebas es adecuado porque no pueden ser separados por una línea recta, lo que requiere de un modelo no lineal.

Nótese que se han generado 100 puntos de ejemplo, lo cual es un conjunto pequeño para una red neuronal y proporciona una mayor probabilidad de overfitting en los datos de entrenamiento, y un error mayor en el conjunto de prueba, lo cual representa un escenario adecuado para el uso de regularización. Además, los ejemplos cuenta con algo de ruido, lo que le da la oportunidad al modelo de aprender aspectos de la muestra que no son generalizables.

Dividamos manualmente el conjunto de datos en train y test:

In [ ]:
n_train = 30
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

### Modelo con sobreajuste

Ilustremos el caso de un modelo con sobreajuste a los datos.

El modelo consistirá de: 
- 500 neuronas en la capa de entrada y una función de activación `relu`.
- En la capa de salida se contará con una función de activación sigmoid y una neurona. 
- La función de perdida será de entropía cruzada binaria.
- El optimizador a usar será `adam`.
- La métrica `accuracy`.


In [ ]:
keras.backend.clear_session()
model = keras.Sequential()
model.add(keras.layers.Dense(500, input_dim = 2, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=4000, verbose = 0)
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print(f"Puntaje de entrenamiento: {train_acc:.3f}")
print(f"Puntaje de evaluación: {test_acc:.3f}")

In [ ]:
plot_decision_boundary(model, X, y)

In [ ]:
DataFrame(history.history).plot()
plt.show()

### Modelo con regularización de pesos:

Se puede adicionar regularización de pesos a la capa de entrada con el fin de reducir el overfitting del modelo a los datos de entrenamiento y mejorar el desempeño en el grupo de test. 

Para la regularización usaremos la penalización en la norma $l2$ con un parámetro de regularización arbitrario $\lambda = 0.001$

In [ ]:
keras.backend.clear_session()
model_l2 = keras.Sequential()
model_l2.add(keras.layers.Dense(500, input_dim = 2, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
model_l2.add(keras.layers.Dense(1, activation='sigmoid'))
model_l2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_l2.summary()

In [ ]:
history_l2 = model_l2.fit(trainX, trainy, validation_data=(testX, testy), epochs=4000, verbose=0)
_, train_acc = model_l2.evaluate(trainX, trainy, verbose=0)
_, test_acc = model_l2.evaluate(testX, testy, verbose=0)

print(f"Puntaje de entrenamiento: {train_acc:.3f}")
print(f"Puntaje de evaluación: {test_acc:.3f}")

In [ ]:
plot_decision_boundary(model_l2, X, y)

In [ ]:
DataFrame(history.history).plot()
plt.show()

### Búsqueda del hiperparámetro óptimo $\lambda$

Una vez se confirma que la regurarización ayuda a disminuir el sobre ajuste al modelo, se puede probar diferentes valores de regularización.

Es una buena práctica buscar algunos valores de $\lambda$ entre 0 y 0.1.

Definamos un ciclo for para hacer estas búsquedas:

In [ ]:
# valores para el gridsearh
lambdas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
train_set, test_set = list(), list()

for val in lambdas:
  print(f"Processing lambda: {val}")
  keras.backend.clear_session()
  model = keras.Sequential()
  model.add(keras.layers.Dense(500, input_dim=2, activation='relu', kernel_regularizer=keras.regularizers.l2(val)))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # Ajuste del modelo
  model.fit(trainX, trainy, epochs=4000, verbose=0)
  # Evaluación del modelo
  _, train_acc = model.evaluate(trainX, trainy, verbose=0)
  _, test_acc = model.evaluate(testX, testy, verbose=0)
  print('Param: %f, Train: %.3f, Test: %.3f' % (val, train_acc, test_acc))
  train_set.append(train_acc)
  test_set.append(test_acc)

In [ ]:
plt.semilogx(lambdas, train_set, label='train', marker='o')
plt.semilogx(lambdas, test_set, label='test', marker='o')
plt.legend()
plt.show()

In [ ]:
print(lambdas)
print(train_set)
print(test_set)

In [ ]:
np.argmin(np.array(train_set)-np.array(test_set))

In [ ]:
lambdas[np.argmin(np.array(train_set)-np.array(test_set))]

## **Dropout**

Es uno de lo métodos mas usados como regularizador en redes neuronales. Fue propuesto por Geoffrey Hinton en el  2012. Dropout consiste en que en cada paso de entrenamiento se quitarán ciertas neuronas de nuestra red, es decir se tendrá cierta probabilidad "**P**" de que una neurona sea quitada o apagada (hacerla cero) durante ese paso del entrenamiento, para esto se tendrá en cuenta la capa de entrada y se excluíra la capa de salida.

<p><img alt="Colaboratory logo" height="150px" src="https://miro.medium.com/max/1354/1*skMXofkjeXtKzSr5lqIEmg.png" align="center" hspace="10px" vspace="0px"></p>

El hiperparámetro "**P**" es llamado **dropout rate**, normalmente tiene un valor entre $0.2$ y $0.5$, es decir, si tomamos 0.2, $1$ de $5$ unidades serán quitadas. Si ciertas neuronas en un paso son desactivadas, al siguiente no necesariamente lo estarán, pues de nuevo se tendrán en cuenta a la hora de hacer la desactivación en el nuevo paso del entrenamiento. Es importante aclarar que el **dropout** solo se realizará durante la etapa de entrenamiento, no en la de validación.

Este método de regularización es altamente usado pues se comporta como un **ensemble**, es decir, como la unión de varios modelos, esto puede ser interpretado de esta manera, pues en cada paso del entrenamiento al tener diferentes neuronas desactivadas, esto "equivale" a tener diferentes redes neuronales. Su implementación en keras es la siguiente

In [ ]:
model = keras.models.Sequential([
                  keras.layers.Flatten(input_shape=[28, 28]),
                  keras.layers.Dropout(rate=0.2),
                  keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
                  keras.layers.Dropout(rate=0.2),
                  keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
                  keras.layers.Dropout(rate=0.2),
                  keras.layers.Dense(10, activation="softmax")
])

Donde en esta implementación vemos que se usó **keras.layers.Dropout**, donde durante el entrenamiento keras quita algunas de las entradas(haciendolas cero). 




## **Regularización de norma máxima (Max-norm)**

Es una forma de regularización muy usada en redes neuronales la cual consiste en restringir los valores de los pesos a cierto valor máximo, **r**, el cual es su hiperparámetro. En este caso no se le añade ningún término a la función de costo, en vez de eso se cálcula $|\textbf{w}|_{2}$ en cada paso y si se supera el valor límite se reescalan los valores de la siguiente forma:
\begin{equation}
\textbf{w} \leftarrow \textbf{w}\frac{r}{|\textbf{w}|_{2}} 
\end{equation}

Reucir el **r** implica incrementar la cantidad de regularización, lo cual puede ayudar con el overfitting. Su implementación en Keras es en la siguiente manera

In [ ]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                    kernel_constraint=keras.constraints.max_norm(1.))

## **Early Stopping**

Este  es un tipo de método de validación cruzada, el cual conciste en analizar el conjunto de validación y en el momento en que su error empiece a aumentar detener el entrenamiento. [callbacks](https://keras.io/callbacks/)

<p><img alt="Colaboratory logo" height="250px" src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/04/Screen-Shot-2018-04-04-at-12.31.56-AM.png" align="center" hspace="10px" vspace="0px"></p>


En keras se puede aplicar este método, haciendo uso de los *callbacks*, los cuales me permiten monitorear él codigo e interactuar con el proceso de entrenamiento de forma automática.

In [ ]:
from keras.callbacks import EarlyStopping

EarlyStopping(monitor='val_err', patience=5)

# Ejercicios

## Ejercicio 1:

Entrene una red neuronal para el dataset iris y realice los siguiente pasos:

1. Carge los datos y estandarícelos.
2. Use la función de keras `to_categorical` para dummificar la variable $y$.
1. 2 capas (contando la capa de entrada) de 16 y 8 neuronas cada una, función de activación ReLU e inicializadando los pesos con he_normal. 
2. Entrene la red usando el optimizador **Adam** y función de perdida `categorical_crossentropy`.

3. Cree los siguientes callbaks y llamelos en el entrenamiento:

```
my_callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='loss', patience=20)]
```


 4. Mediante el parámetro del método `fit` `validation_split`, usando un 20% como datos de validación. 
 5. Defina 1000 épocas de entrenamiento.
 6. Fije el parámetro de `batch_size` en 16.
 7. Entrene el modelo y guarde el entrenamiento en un objeto `history`.
 8. Grafique las curvas de entrenamiento y validación.

In [ ]:
# Use estas funciones y librerías.
from sklearn.datasets import load_iris
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from pandas import DataFrame
import matplotlib.pyplot as plt

In [ ]:
#Celda para carga de datos y estandarización.

In [ ]:
#@title Si tiene problemas para cargar los datos y estandarizarlos, haga click en Mostrar código.
data=load_iris()
X,y=data.data,data.target
X-=X.mean(axis=0)
X/=X.std(axis=0)
y=to_categorical(y)

In [ ]:
#Celda para creación del modelo

In [ ]:
#@title Si tiene problemas para crear el modelo, haga click en Mostrar código.
keras.backend.clear_session()
modelo=Sequential()
modelo.add(Dense(16,input_dim=4,activation='relu',kernel_initializer='he_normal'))
modelo.add(Dense(8,activation='relu',kernel_initializer='he_normal'))
modelo.add(Dense(3,activation='softmax',kernel_initializer='glorot_normal'))
modelo.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Definición de los callbacks

In [ ]:
#@title Si tiene  problema para definir los callbacks, haga click en Mostrar Código.
my_callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='loss', patience=20)]

In [ ]:
# Ajuste del modelo

In [ ]:
#@title Si tiene problema para definir el ajuste del modelo, haga click en Mostrar código.
history=modelo.fit(x=X, y=y, validation_split=0.2, epochs=1000, callbacks=my_callbacks, batch_size=16)

In [ ]:
# Gráfica

In [ ]:
#@title Si tiene problemas para graficar, haga click en Mostrar código.
DataFrame(history.history).plot(figsize=(10,10))
plt.grid(True)
plt.show()

## Ejercicio 2:

Realice el mismo ejercicio anterior agregando una capa de `dropout` del 20% entre las capas de 16 y 8 neuronas.

¿Qué conclusión puede sacar?

In [ ]:
# Celda para definición del modelo

In [ ]:
#@title Si tiene problema para definir el modelo, haga click en Mostrar código.
keras.backend.clear_session()
modelo=Sequential()
modelo.add(Dense(16,input_dim=4,activation='relu',kernel_initializer='he_normal'))
modelo.add(Dropout(0.2))
modelo.add(Dense(8,activation='relu',kernel_initializer='he_normal'))
modelo.add(Dense(3,activation='softmax',kernel_initializer='glorot_normal'))
modelo.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Celda para definir los callbacks

In [ ]:
#@title Si tiene problemas para definir los callbacks, haga click en Mostrar código.
my_callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='accuracy', patience=100),
                keras.callbacks.EarlyStopping(monitor='loss', patience=20)]

In [ ]:
# Celda para el entrenamiento del modelo.

In [ ]:
#@title Si tiene problema para entrenar el modelo, haga click en Mostrar código.
history=modelo.fit(x=X,y=y,validation_split=0.2,epochs=1000, callbacks=my_callbacks, batch_size=16)

In [ ]:
# Celda para graficar

In [ ]:
#@title Si tiene problemas para graficar, haga click en Mostrar código.
DataFrame(history.history).plot(figsize=(10,10))
plt.grid(True)
plt.show()